## Notbook for calculating the Global Windatlas parts to be downloaded, conforming with the 250.000 km**2 restrictions

In [67]:
import math
from shapely.geometry import LineString, MultiPolygon, Polygon, box
from shapely.ops import split

import geopandas as gpd
import xarray as xr
import rioxarray as rioxr 
import rasterio as rio

from utils import geo, data

import cartopy.io.shapereader as shpreader

import yaml

from rioxarray.merge import merge_arrays

In [8]:
ROOT = 'C:/CaT/Masterthesis/repository/'
LUISA = ROOT + 'data/input_geodata/luisa.tif'

In [52]:
def gwa_bounds_prep(country, padding, shape_filter, savepath = ROOT + 'data/country_gwa_bounds/'):
    
    #construct bbox df with adjusted bounds for full era5 overlap
    bounds = country.to_crs(3035).bounds.values[0]
    bb = box(bounds[0]-padding, bounds[1]-padding, bounds[2]+padding, bounds[3]+padding)
    bb_df = gpd.GeoDataFrame(gpd.GeoSeries(bb), columns=['geometry'], crs = 3035)
    bb_df["area"] = bb_df['geometry'].area/ 10**6
    
    #identify needed parts according to GWA 250000 km**2 limitation
    parts = math.ceil(bb_df['area']/250000)
    
    if parts == 1:
        bb_df.to_crs(4326).to_file(savepath + f'{cntry}.shp')
        return parts
    else:
        minx, miny, maxx, maxy = bb.bounds

        dx = (maxx - minx) / parts # width of a small part

        vertical_splitters = [LineString([(minx + i*dx, miny), (minx + i*dx, maxy)]) for i in range(parts)]
                      
        bb_split = bb
        for splitter in vertical_splitters:
            bb_split = MultiPolygon(split(bb_split, splitter))
                      
        for i in range(len(bb_split.geoms)):
            sp = savepath + f'{cntry}_part_{str(i+1)}.shp'
            gpd.GeoDataFrame([bb_split.geoms[i]], columns=['geometry'], crs = 3035).to_crs(4326).to_file(sp)
            
        return parts
            

### 1. Bounds part prep

In [53]:
with open("countrycode.yaml", 'r') as stream:
    countrycode = yaml.safe_load(stream)  
    
countries = countrycode.keys()

country = gpd.read_file(ROOT + f'data/country_shapes/{cntry}.gpkg')

In [54]:
part_d = {}

for cntry in countries:
    country = gpd.read_file(ROOT + f'data/country_shapes/{cntry}.gpkg')
    part_d[cntry] =  gwa_bounds_prep(country, padding=30000, shape_filter = LUISA)

In [73]:
part_d2plus = {k:v for k,v in zip(part_d.keys(), part_d.values()) if not v == 1}

list(range(1,3))

[1, 2]

### 2. Stiching together downloaded gwa.tif files

In [76]:
for cntry in ['Germany']:
    ls = []
    
    for i in range(part_d2plus[cntry]):
    
        with rioxr.open_rasterio(ROOT + f'data/gwa/{cntry}_part_{i+1}_wind-speed_100m.tif') as f:
            gwa = f
            f.close()
        ls.append(gwa)
        
    country_gwa = merge_arrays(ls, nodata = 0)
    country_gwa.rio.to_raster(ROOT + f'data/gwa/Germany_wind-speed_100m.tif', recalc_transform = True)